# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f80589a2970>
├── 'a' --> tensor([[-0.2047,  1.9863, -1.2117],
│                   [-1.0611, -0.5134, -0.5869]])
└── 'x' --> <FastTreeValue 0x7f80589a2d00>
    └── 'c' --> tensor([[ 0.9535, -0.5228,  0.4578, -0.3437],
                        [-0.4735,  1.7184, -0.6564,  0.2577],
                        [ 0.1180,  0.5349, -0.5621, -0.8449]])

In [4]:
t.a

tensor([[-0.2047,  1.9863, -1.2117],
        [-1.0611, -0.5134, -0.5869]])

In [5]:
%timeit t.a

61.9 ns ± 0.0301 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f80589a2970>
├── 'a' --> tensor([[-1.4199, -0.8898,  1.1204],
│                   [ 0.5714,  0.4129, -1.5237]])
└── 'x' --> <FastTreeValue 0x7f80589a2d00>
    └── 'c' --> tensor([[ 0.9535, -0.5228,  0.4578, -0.3437],
                        [-0.4735,  1.7184, -0.6564,  0.2577],
                        [ 0.1180,  0.5349, -0.5621, -0.8449]])

In [7]:
%timeit t.a = new_value

63.8 ns ± 0.0499 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2047,  1.9863, -1.2117],
               [-1.0611, -0.5134, -0.5869]]),
    x: Batch(
           c: tensor([[ 0.9535, -0.5228,  0.4578, -0.3437],
                      [-0.4735,  1.7184, -0.6564,  0.2577],
                      [ 0.1180,  0.5349, -0.5621, -0.8449]]),
       ),
)

In [10]:
b.a

tensor([[-0.2047,  1.9863, -1.2117],
        [-1.0611, -0.5134, -0.5869]])

In [11]:
%timeit b.a

58.3 ns ± 0.0215 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.8038, -0.3294, -1.2446],
               [ 1.0708,  0.2104, -1.2126]]),
    x: Batch(
           c: tensor([[ 0.9535, -0.5228,  0.4578, -0.3437],
                      [-0.4735,  1.7184, -0.6564,  0.2577],
                      [ 0.1180,  0.5349, -0.5621, -0.8449]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 0.0968 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

830 ns ± 0.186 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.1 µs ± 39.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 855 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 427 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7fab668f40>
├── 'a' --> tensor([[[-0.2047,  1.9863, -1.2117],
│                    [-1.0611, -0.5134, -0.5869]],
│           
│                   [[-0.2047,  1.9863, -1.2117],
│                    [-1.0611, -0.5134, -0.5869]],
│           
│                   [[-0.2047,  1.9863, -1.2117],
│                    [-1.0611, -0.5134, -0.5869]],
│           
│                   [[-0.2047,  1.9863, -1.2117],
│                    [-1.0611, -0.5134, -0.5869]],
│           
│                   [[-0.2047,  1.9863, -1.2117],
│                    [-1.0611, -0.5134, -0.5869]],
│           
│                   [[-0.2047,  1.9863, -1.2117],
│                    [-1.0611, -0.5134, -0.5869]],
│           
│                   [[-0.2047,  1.9863, -1.2117],
│                    [-1.0611, -0.5134, -0.5869]],
│           
│                   [[-0.2047,  1.9863, -1.2117],
│                    [-1.0611, -0.5134, -0.5869]]])
└── 'x' --> <FastTreeValue 0x7f8034a63be0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.8 µs ± 91.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f802ff731c0>
├── 'a' --> tensor([[-0.2047,  1.9863, -1.2117],
│                   [-1.0611, -0.5134, -0.5869],
│                   [-0.2047,  1.9863, -1.2117],
│                   [-1.0611, -0.5134, -0.5869],
│                   [-0.2047,  1.9863, -1.2117],
│                   [-1.0611, -0.5134, -0.5869],
│                   [-0.2047,  1.9863, -1.2117],
│                   [-1.0611, -0.5134, -0.5869],
│                   [-0.2047,  1.9863, -1.2117],
│                   [-1.0611, -0.5134, -0.5869],
│                   [-0.2047,  1.9863, -1.2117],
│                   [-1.0611, -0.5134, -0.5869],
│                   [-0.2047,  1.9863, -1.2117],
│                   [-1.0611, -0.5134, -0.5869],
│                   [-0.2047,  1.9863, -1.2117],
│                   [-1.0611, -0.5134, -0.5869]])
└── 'x' --> <FastTreeValue 0x7f7fab73a220>
    └── 'c' --> tensor([[ 0.9535, -0.5228,  0.4578, -0.3437],
                        [-0.4735,  1.7184, -0.6564,  0.2577],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 32.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.6 µs ± 83.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.9535, -0.5228,  0.4578, -0.3437],
                       [-0.4735,  1.7184, -0.6564,  0.2577],
                       [ 0.1180,  0.5349, -0.5621, -0.8449]],
              
                      [[ 0.9535, -0.5228,  0.4578, -0.3437],
                       [-0.4735,  1.7184, -0.6564,  0.2577],
                       [ 0.1180,  0.5349, -0.5621, -0.8449]],
              
                      [[ 0.9535, -0.5228,  0.4578, -0.3437],
                       [-0.4735,  1.7184, -0.6564,  0.2577],
                       [ 0.1180,  0.5349, -0.5621, -0.8449]],
              
                      [[ 0.9535, -0.5228,  0.4578, -0.3437],
                       [-0.4735,  1.7184, -0.6564,  0.2577],
                       [ 0.1180,  0.5349, -0.5621, -0.8449]],
              
                      [[ 0.9535, -0.5228,  0.4578, -0.3437],
                       [-0.4735,  1.7184, -0.6564,  0.2577],
                       [ 0.1180,  0.5349, -0.5621, -0.8449]],

In [26]:
%timeit Batch.stack(batches)

78.4 µs ± 92.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.9535, -0.5228,  0.4578, -0.3437],
                      [-0.4735,  1.7184, -0.6564,  0.2577],
                      [ 0.1180,  0.5349, -0.5621, -0.8449],
                      [ 0.9535, -0.5228,  0.4578, -0.3437],
                      [-0.4735,  1.7184, -0.6564,  0.2577],
                      [ 0.1180,  0.5349, -0.5621, -0.8449],
                      [ 0.9535, -0.5228,  0.4578, -0.3437],
                      [-0.4735,  1.7184, -0.6564,  0.2577],
                      [ 0.1180,  0.5349, -0.5621, -0.8449],
                      [ 0.9535, -0.5228,  0.4578, -0.3437],
                      [-0.4735,  1.7184, -0.6564,  0.2577],
                      [ 0.1180,  0.5349, -0.5621, -0.8449],
                      [ 0.9535, -0.5228,  0.4578, -0.3437],
                      [-0.4735,  1.7184, -0.6564,  0.2577],
                      [ 0.1180,  0.5349, -0.5621, -0.8449],
                      [ 0.9535, -0.5228,  0.4578, -0.3437],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 336 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

311 µs ± 2.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
